<?xml version="1.0" encoding="utf-8"?>
<rfcOpenApi>
 <header>
  <resultCode>
   00
  </resultCode>
  <resultMsg>
   success
  </resultMsg>
 </header>
 <body>
  <pageIndex>
   0
  </pageIndex>
  <pageSize>
   2000
  </pageSize>
  <startPage>
   1
  </startPage>
  <totalCount>
   78
  </totalCount>
  <data>
   <list>
    <baseDate>
     2020-08-31
    </baseDate>
    <cameraCnt>
     1.0
    </cameraCnt>
    <cameraPixel>
     41.0
    </cameraPixel>
    <cctvAddr>
    </cctvAddr>
    <cctvNm>
     교통정보수집
    </cctvNm>
    <cctvPosx>
     127.1214007
    </cctvPosx>
    <cctvPosy>
     35.869711
    </cctvPosy>
    <dongAddr>
     전라북도 전주시 덕진구 송천동 437-1
    </dongAddr>
    <insYM>
     Jan-97
    </insYM>
    <manOfficial>
     전라북도 전주시청
    </manOfficial>
    <manTel>
     063-281-2728
    </manTel>
    <shootDir>
     360도 전방면
    </shootDir>
    <sid>
     1
    </sid>
    <storageDay>
     30.0
    </storageDay>
   </list>
   <list>
    <baseDate>
     2020-08-31
    </baseDate>
    